In [4]:
! pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 78.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.1 MB/s eta 0:00:00


In [11]:
from onnxruntime.quantization import quantize_dynamic, QuantType
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

In [13]:
model_path = "/kaggle/input/amazon_review_distilbert/transformers/default/1" 
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)


model.eval()


dummy_input = tokenizer("This is a dummy input.", return_tensors="pt")


onnx_path = "distilbert_onnx.onnx"


torch.onnx.export(
    model,
    tuple(dummy_input.values()),  
    onnx_path,  
    opset_version=14, 
    input_names=["input_ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={  
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "logits": {0: "batch_size"},
    },
)

print(f"ONNX model exported to: {onnx_path}")

ONNX model exported to: distilbert_onnx.onnx


In [14]:
onnx_model_path = "/kaggle/working/distilbert_onnx.onnx"
quantized_model_path = "distilbertmodel_quantized.onnx"

# Apply dynamic quantization
quantize_dynamic(
    model_input=onnx_model_path, 
    model_output=quantized_model_path, 
    weight_type=QuantType.QInt8 
)

print("Quantization complete! Model saved at:", quantized_model_path)

Quantization complete! Model saved at: distilbertmodel_quantized.onnx
